# figures galore

results visualizations for paper or something idk

## setup, data loading needed for most visualizations

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import json
import math
import os
from pathlib import Path
from typing import List
import sys

import cv2
import dataclass_array as dca
import jax.numpy as jnp
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
from PIL import Image
import pandas as pd
import pycolmap
import pyrender
import trimesh
import visu3d as v3d
import yaml

sys.path.append(os.path.abspath(os.path.join("..", "bop_toolkit")))
from bop_toolkit.bop_toolkit_lib.pose_error import vsd, mssd, mspd
from bop_toolkit.bop_toolkit_lib.misc import get_symmetry_transformations
from bop_toolkit.bop_toolkit_lib.renderer import create_renderer

import burybarrel.colmap_util as cutil
from burybarrel.image import render_v3d, render_models, to_contour
from burybarrel.camera import RadialCamera, load_v3dcams
from burybarrel.mesh import subdivide_mesh, recolor_mesh
from burybarrel.transform import T_from_blender, T_from_translation, scale_T_translation, scale_cams, scale_pc
from burybarrel.plotting import get_line3d_trace

In [ ]:
tmpdir = Path("/scratch/jeyan/barreldata/tmp")
metricscsvpath = Path("/scratch/jeyan/barreldata/results/allmetrics.csv")
metricsdf = pd.read_csv(metricscsvpath)
metricsdf.head()

In [ ]:
metricsdf[["avg_vsd", "avg_mssd", "avg_mspd", "multiview_fitted", "pose_type", "use_icp"]].groupby(["multiview_fitted", "pose_type", "use_icp"]).mean()

In [ ]:
# loading EVERYTHING
datasetname = "barrelddt1"
singledatarows = metricsdf[metricsdf["dataset"] == datasetname]

resrow = singledatarows.iloc[3]
datadir = Path(resrow["datadir"])
resdir = Path(resrow["resdir"])
object_name = resrow["object_name"]

with open("../configs/blender_gt_info.yaml", "rt") as f:
    gtinfo = yaml.safe_load(f)
with open(datadir / "gt_obj2cam.json", "rt") as f:
    gtposes = yaml.safe_load(f)
with open(datadir / "info.json", "rt") as f:
    datainfo = yaml.safe_load(f)
with open(datadir / "camera.json", "rt") as f:
    caminfo = yaml.safe_load(f)
with open(resdir / "../../sam-masks/masksinfo.json", "rt") as f:
    masksinfo = yaml.safe_load(f)
posepath = resdir / "estimated-poses.json"
estinfopath = resdir / "reconstruction-info.json"
with open(posepath, "rt") as f:
    ests = yaml.safe_load(f)
with open(estinfopath, "rt") as f:
    estinfo = yaml.safe_load(f)
with open(resdir / "../../foundpose-output/inference/estimated-poses.json", "rt") as f:
    foundposeests = yaml.safe_load(f)
cams, imgpaths = load_v3dcams(resdir / "../../colmap-out/cam_poses.json", img_parent=datadir / "rgb")

densepath = resdir / "../../openmvs-out/scene_dense_trimeshvalid.ply"
densetrimesh: trimesh.PointCloud = trimesh.load_mesh(densepath)
densepc = v3d.Point3d(p=densetrimesh.vertices, rgb=densetrimesh.colors[:, :3])

camspec = RadialCamera.from_jsonargs(**caminfo)
objpath = Path("/scratch/jeyan/barreldata/models3d") / object_name
mesh: trimesh.Trimesh = trimesh.load_mesh(objpath)
meshpc = v3d.Point3d(p=mesh.vertices, rgb=[255, 0, 0])

## some 3d overlay wireframe visualization thing

In [ ]:
idx = 5
est = ests[idx]
gt = gtposes[idx]
est

In [ ]:
meshcol = np.array([255, 0, 0, 255], dtype=np.uint8)
planecol = np.array([0, 0, 255, 255], dtype=np.uint8)
black = np.array([0, 0, 0, 255], dtype=np.uint8)
transparent = np.array([0, 0, 0, 0], dtype=np.uint8)
mesh = recolor_mesh(mesh, meshcol)
plane = trimesh.creation.box(extents=(100, 100, 0.01))
plane = subdivide_mesh(plane, 7)
plane = recolor_mesh(plane, planecol)

In [ ]:
obj2cam = v3d.Transform(R=est["R"], t=est["t"])
floor2cam = v3d.Transform(R=est["R_floor"], t=est["t_floor"])
obj2camgt = v3d.Transform(R=gt["R"], t=gt["t"])
floor2camgt = v3d.Transform(R=gt["R_floor"], t=gt["t_floor"])

In [ ]:
cam = v3d.Camera(spec=camspec, world_from_cam=v3d.Transform.identity())

In [ ]:
maskrgb, _, _ = render_models(cam, [mesh, plane], [obj2cam, floor2cam], light_intensity=1000, flags=pyrender.RenderFlags.FLAT)
objmask = np.all(maskrgb == [255, 0, 0], axis=-1)
rgb_wireframe, _, _ = render_models(cam, [recolor_mesh(plane, black)], [floor2cam], light_intensity=1000, wireframe=True, flags=pyrender.RenderFlags.RGBA)
rgb_wireframe = np.copy(rgb_wireframe)
rgb_wireframe *= ~objmask[..., None]
rgb, _, _ = render_models(cam, [mesh, plane], [obj2cam, floor2cam], light_intensity=50, wireframe=False)
Image.fromarray(rgb_wireframe).save(tmpdir / "rendered_wireframe.png")
Image.fromarray(rgb).save(tmpdir / "rendered.png")
img = cv2.cvtColor(cv2.imread(est["img_path"]), cv2.COLOR_BGR2RGB)
alpha = 0.5
img = cv2.addWeighted(img, alpha, rgb, 1 - alpha, 0)
wireframewhere = np.where(rgb_wireframe[..., 3] > 0)
img[wireframewhere] = rgb_wireframe[wireframewhere][..., :3]
Image.fromarray(img).save(tmpdir / "rendered_overlay.png")

# rgb, _, _ = render_models(cam, [mesh, plane], [obj2camgt, floor2camgt], light_intensity=1000, wireframe=True, flags=pyrender.RenderFlags.RGBA)
# Image.fromarray(rgb).save(tmpdir / "rendered_gt.png")
# img = cv2.cvtColor(cv2.imread(est["img_path"]), cv2.COLOR_BGR2RGB)
# wireframewhere = np.where(rgb[..., 3] > 0)
# img[wireframewhere] = rgb[wireframewhere][..., :3]
# Image.fromarray(img).save(tmpdir / "rendered_overlay_gt.png")

In [ ]:
what = v3d.Transform.from_matrix(np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 4.0],
    [0, 0, 0, 1]
]))
test, _, _ = render_models(cam, [recolor_mesh(plane, [255, 0, 0, 255])], [what], light_intensity=1000, wireframe=True, flags=pyrender.RenderFlags.NONE)
Image.fromarray(test).save(Path("/scratch/jeyan/barreldata/tmp") / "rendered_test.png")

## visualizing mask

In [ ]:
idx = 1
singlemasksinfo = masksinfo[idx]
singlemasksinfo

In [ ]:
bestidx = singlemasksinfo["best_idx"]
# [x_min, y_min, x_max, y_max]
bbox = list(map(int, singlemasksinfo["bboxes"][bestidx]))
mask = np.array(Image.open(singlemasksinfo["mask_paths"][bestidx]).convert("L"))
maskbool = mask > 0
img = np.array(Image.open(singlemasksinfo["img_path"]).convert("RGBA"))

In [ ]:
maskalpha = np.zeros((*img.shape[:2], 4), dtype=np.uint8)
maskalpha[maskbool, 0:3] = [255, 0, 0]
maskalpha[maskbool, 3] = 255
maskalpha[~maskbool] = img[~maskbool]
alpha = 0.5
overlayed = cv2.addWeighted(img, alpha, maskalpha, 1 - alpha, 0)
overlayed = cv2.rectangle(overlayed, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 197, 71, 255), 10)
Image.fromarray(overlayed).save(tmpdir / "rendered_overlay_mask.png")

## multiview visualization thing

In [ ]:
# Tworld_obj = T_from_blender(gtinfo[datasetname]["R"], gtinfo[datasetname]["t"], scale=gtinfo[datasetname]["scalefactor"])
scale = 1 / gtinfo[datasetname]["scalefactor"]
T_zup = T_from_blender(gtinfo[datasetname]["R_floor"], gtinfo[datasetname]["t_floor"], scale=1).inv

In [ ]:
datainfo

In [ ]:
gtposes[0]

In [ ]:
densepc_zup = scale_pc(T_zup @ densepc, scale)
cams_zup = scale_cams(cams.apply_transform(T_zup), scale)
cam1 = cams_zup[0]
cam2 = cams_zup[-1]
foundposeest1 = list(filter(lambda x: Path(x["img_path"]).stem == imgpaths[0].stem, foundposeests))[2]
foundposeT1 = v3d.Transform(R=foundposeest1["R"], t=foundposeest1["t"])
v3d.make_fig([densepc_zup, cam1, cam2, get_line3d_trace(cams_zup.world_from_cam.t, markersize=3, linewidth=3), cam1.world_from_cam @ foundposeT1 @ meshpc])

In [ ]:
img = np.array(Image.open(imgpaths[0]).convert("RGBA"))
T_objworld = cam1.world_from_cam @ foundposeT1
objwireframe, _, _ = render_models(cam1, [recolor_mesh(mesh, [255, 0, 0, 255])], [T_objworld], light_intensity=5000, wireframe=True, flags=pyrender.RenderFlags.RGBA)
wireframemask = objwireframe[..., 3] > 0
objwireframe[~wireframemask] = img[~wireframemask]
Image.fromarray(objwireframe)

In [ ]:
img = np.array(Image.open(imgpaths[-1]).convert("RGBA"))
objwireframe, _, _ = render_models(cam2, [recolor_mesh(mesh, [255, 0, 0, 255])], [T_objworld], light_intensity=5000, wireframe=True, flags=pyrender.RenderFlags.RGBA)
wireframemask = objwireframe[..., 3] > 0
objwireframe[~wireframemask] = img[~wireframemask]
Image.fromarray(objwireframe)